In [2]:
install.packages("tidyverse")
install.packages("rmarkdown")
install.packages("ggplot2") 
install.packages("caret")
install.packages("devtools")
install.packages("dplyr")
install.packages("rJava",,"http://rforge.net/",type="source")
install.packages("RWeka")
install.packages("infotheo")
install.packages("pROC")
install.packages("reshape2")
install.packages("corrplot")
install.packages("Hmisc") 
install.packages("Matrix") 
install.packages("randomForest") 
install.packages("glmnet") 
install.packages("gbm")
install.packages("e1071")
install.packages("pamr")

source("https://bioconductor.org/biocLite.R")
biocLite("pathifier")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
also installing the dependencies ‘ini’, ‘clisymbols’, ‘fs’, ‘gh’, ‘rprojroot’, ‘rex’, ‘crosstalk’, ‘xopen’, ‘brew’, ‘commonmark’, ‘praise’, ‘usethis’, ‘covr’, ‘desc’, ‘DT’, ‘git2r’, ‘memoise’, ‘pkgbuild’, ‘pkgload’, ‘rcmdcheck’, ‘remotes’, ‘roxygen2’, ‘rversions’, ‘sessioninfo’, ‘testthat’

Warning message in install.packages("devtools"):
“installation of package ‘fs’ had non-zero exit status”Warning message in install.packages("devtools"):
“installation of package ‘git2r’ had non-zero exit status”Warning message in install.packages("devtools"):
“installation of package ‘usethis’ had non-zero exit status”Warning message in install.packages("devtools"):
“installation of package ‘d

ERROR: Error: With R version 3.5 or greater, install Bioconductor packages using BiocManager; see https://bioconductor.org/install


In [1]:
getwd()

[1] "/Users/rrrrrita/Downloads/lilikoi-master"

#### Lilikoi: an R package of personalized metabolomics pathway analysis
#### Lilikoi is a novel tool for personalized pathway analysis of metabolomics data. 
#### It has four modules:
* feature mapper, dimension transformation, feature selection, and classification and prediction.
<img src="inst/extdata/FIG1.JPG">

In [8]:
install.packages("devtools", "/home/rrrrrita/anaconda3/lib/R/library")

Warning message in install.packages("devtools", "/home/rrrrrita/anaconda3/lib/R/library"):
“'lib = "/home/rrrrrita/anaconda3/lib/R/library"' is not writable”

ERROR: Error in install.packages("devtools", "/home/rrrrrita/anaconda3/lib/R/library"): unable to install packages


In [12]:
 
devtools::install_github("FADHLyemen/lilikoi_Fadhl")
# library(lilikoi)

Warning message in system(cmd):
“error in running command”Warning message in utils::untar(tarfile, ...):
“‘/usr/local/bin/gtar -xf '/var/folders/jh/qnytjxpx13j0l4yl2b8j83wm0000gn/T//RtmpEKFlsW/file3cf64472349d.tar.gz' -C '/var/folders/jh/qnytjxpx13j0l4yl2b8j83wm0000gn/T//RtmpEKFlsW/remotes3cf625dc5286'’ returned error code 127”

ERROR: Error in system(cmd, intern = TRUE): error in running command


## Data format: 
### User recommends to save their data in csv format. Columns are metabolites and rows are samples. Case and control label should be saved in the second column such as the below raw data:
#### Note: make sure to disable the check.names option in the read.csv

In [10]:
data_dir=system.file("extdata", "plasma_breast_cancer.csv", package = "lilikoi")
plasma_data=read.csv(data_dir,check.names=F,row.names=1)
head(plasma_data)
dim(plasma_data)
table(plasma_data$Label)

Warning message in file(file, "rt"):
“file("") only supports open = "w+" and open = "w+b": using the former”

ERROR: Error in read.table(file = file, header = header, sep = sep, quote = quote, : no lines available in input


## The above data set is the metabolomics cohort which  is composed of 126 breast cancer and 81 control plasma samples from City of Hope Hospital (COH) 
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4818393/

## I- Load the data using Lilikoi Loaddata function

In [ ]:
Loaddata(file=system.file("extdata", "plasma_breast_cancer.csv", package = "lilikoi"))

## II- Transforms the metabolite names to the HMDB ids uisng Lilikoi MetaTOpathway function
### Lilikoi allows the user to input any kind of metabolite IDs including metabolites names ('name') along with synonyms, KEGG IDs ('kegg'), HMDB IDs ('hmdb') and PubChem IDs ('pubchem'). 
### if the metabolites have a standard names as ID, Lilikoi will match these names among 100 k saved database, if there are not any hits, Lilikoi will perform fuzzy matching to find the closest matching for this metabolite.
### The below flowchart will explain this matching process in more details.
<img src="inst/extdata/matching.JPG">

In [ ]:
Metabolite_pathway_table=MetaTOpathway('name')
head(Metabolite_pathway_table)

In [ ]:
Query

## III- Transform metabolites into pathway using Pathifier algorithm
https://www.bioconductor.org/packages/release/bioc/html/pathifier.html
#### A specific pathway dysregulation score (PDS) is inferred to measure the abnormity for each sample in each pathway. For each pathway, the samples are mapped in a high dimensional principal component space and a principal curve is constructed along the samples and smoothed. The PDS score measures the distance from the projected dot along the curve to the centroid of normal samples (origin point of the curve).

In [ ]:
PDSmatrix= PDSfun(Metabolite_pathway_table)

### Using PDSfun, we generate a new matrix which has pathways as columns instead of metaboltes. 

In [ ]:
head(t(PDSmatrix))
dim(t(PDSmatrix))

## IV- Select the most signficant pathway related to phenotype.
### We used two methods for pathway selection which are implemented in Weka package:
https://cran.r-project.org/web/packages/RWeka/RWeka.pdf

* information gain ('info') and 
* gain ratio ('gain')

### If you did not get any selected pathwys, you can lower the threshold below 0.5

In [ ]:
selected_Pathways_Weka= featuresSelection(PDSmatrix,threshold= 0.54,method="gain")
selected_Pathways_Weka

## V- Classification and prediction 
### This function will randomly separate the PDS score matrix with only the selected pathways into training and testing sets. It will use seven widely used machine learning algorithms to build the classification model from the training set. It plots the pathway importance from each model and its accuracy (AUC, sensitivity, specificity). 

In [ ]:
result=machine_learning(PDSmatrix,selected_Pathways_Weka);

# VI- Model Adjustemnt using clinical factors such as Age and ethnicity
## This function include the clinical factors included by the users and comput model performance using these factors. 
* Model will be created using only the best performed model from the last step
* Make sure to include the metadata for each samples 

In [ ]:
clinical_factors_data=read.csv(file=system.file("extdata", "plasma_breast_cancer_Meta.csv", package = "lilikoi")) 
head(clinical_factors_data)
str(clinical_factors_data)

In [ ]:
lilikoi::model_adjustment(result,PDSmatrix,selected_Pathways_Weka,clinical_factors_data,factors=c('Age','Race'))